# COGS 118A- Project Checkpoint

# Names

- Drake Coy
- Aditi Krishnakumar
- Simran Nayyer
- Jasmine Qiang
- Luke Sztajnkrycer

# Abstract 
Our project is centered around accurately predicting the price of flights to find the best booking times and routes. Our goal is to enable customers to plan their trips as efficiently and cost effective as possible. We will examine data collected from ‘Easemytrip,’ an online booking platform, between February 11th and March 31st of 2022 that includes flight information from India about airline companies, flight codes, source/destination cities, flight times, and more. There are 12 total features that mostly take the form of discrete categorical variables (‘Delhi,’ ‘morning,’ ‘economy,’ etc.), and includes 300,000 unique observations. We plan on implementing both a decision tree and linear regression model to make our predictions. Each method will be compared to evaluate which provides a higher success rate. We will measure the performance of our linear regression model by utilizing both a R-squared and adjusted R-squared metric to evaluate. We will then evaluate our decision tree success rate by using K-means cross validation (using ~⅕ of the data as a test set) to determine how generalizable our decision criteria are to any given portion of the data.

# Background

Fill in the background and discuss the kind of prior work that has gone on in this research area here. **Use inline citation** to specify which references support which statements.  You can do that through HTML footnotes (demonstrated here). I used to reccommend Markdown footnotes (google is your friend) because they are simpler but recently I have had some problems with them working for me whereas HTML ones always work so far. So use the method that works for you, but do use inline citations.

Here is an example of inline citation. After government genocide in the 20th century, real birds were replaced with surveillance drones designed to look just like birds<a name="lorenz"></a>[<sup>[1]</sup>](#lorenznote). Use a minimum of 2 or 3 citations, but we prefer more <a name="admonish"></a>[<sup>[2]</sup>](#admonishnote). You need enough citations to fully explain and back up important facts. 

Remeber you are trying to explain why someone would want to answer your question or why your hypothesis is in the form that you've stated. 

# Problem Statement

Clearly describe the problem that you are solving. Avoid ambiguous words. The problem described should be well defined and should have at least one ML-relevant potential solution. Additionally, describe the problem thoroughly such that it is clear that the problem is quantifiable (the problem can be expressed in mathematical or logical terms), measurable (the problem can be measured by some metric and clearly observed), and replicable (the problem can be reproduced and occurs more than once).

# Data

UPDATED FROM PROPOSAL!

You should have obtained and cleaned (if necessary) data you will use for this project.

Please give the following infomration for each dataset you are using
- link/reference to obtain it
- description of the size of the dataset (# of variables, # of observations)
- what an observation consists of
- what some critical variables are, how they are represented
- any special handling, transformations, cleaning, etc you have done should be demonstrated here!


# Proposed Solution

In this section, clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. Provide enough detail (e.g., algorithmic description and/or theoretical properties) to convince us that your solution is applicable. Make sure to describe how the solution will be tested.  

If you know details already, describe how (e.g., library used, function calls) you plan to implement the solution in a way that is reproducible.

If it is appropriate to the problem statement, describe a benchmark model<a name="sota"></a>[<sup>[3]</sup>](#sotanote) against which your solution will be compared. 

# Evaluation Metrics

Since we are using a regression model to predict continuous data (price), our primary evaluation metric will be an R Square Score which indicates the percentage of variance in the dependent variable that is explained by the independent variables (i.e. how well the model fits our dependent variable (price)). It is a score between 0 and 1 where 1 indicates a perfect fit between predicted and actual values. 

The numerator contains the sum of the residuals squared and the denominator is the sum squared of the distance between the data and the mean.

Since R Square does not account for the problem of overfitting, we will additionally look at Adjusted R Square which penalizes additional independent variables added to the model and adjusts the evaluation metric to account for overfitting. It is always less than or equal to the R Square score. 

N is the number of sample data points and k is the number of independent variables in the model.

# Preliminary results

NEW SECTION!

Please show any preliminary results you have managed to obtain.

Examples would include:
- Analyzing the suitability of a dataset or alogrithm for prediction/solving your problem 
- Performing feature selection or hand-designing features from the raw data. Describe the features available/created and/or show the code for selection/creation
- Showing the performance of a base model/hyper-parameter setting.  Solve the task with one "default" algorithm and characterize the performance level of that base model.
- Learning curves or validation curves for a particular model
- Tables/graphs showing the performance of different models/hyper-parameters



# Ethics & Privacy

We have chosen this particular dataset as it minimizes the exposure of personally identifiable information, containing only details about the airline itself. Furthermore, this data is sourced from a publicly available website ‘Easemytrip’ and published on Kaggle, which provides users consent to explore, publish, and access the information. 


Our model itself is used to predict the price alone and nothing that may cause ethical concerns once in production such as the specific flight route, passenger details, etc. Additionally, the data is based on previous flight routes alone. 


We do acknowledge the possible bias in our dataset because all the flight routes are within a single country (India). This is a limitation in the generalizability of our model and will therefore limit its uses in production. We will ensure that this bias is highlighted to the users.

# Team Expectations 

* *Tell the team ahead of time if you can't make a meeting. Create a plan to get caught up on anything missed.*
* *Finish your work done before any applicable meeting so that it can be discussed with the group.*
* *Coordinate with members for your part of the assignment.*
* *Communicate respectfully and punctually.*

__Project Checkpoint Responsibilities:__  
Luke - import data sets, clean data and prepare for EDA/model selection  
Aditi, Simran, Jasmine - preform EDA  
Drake - begin model selection and looking and the most important features in the dataset  

# Project Timeline Proposal

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 4/20  |  3 PM |  Select project idea and dataset, then divide up project proposal work among the group.  | Select project idea and dataset, then divide up project proposal work among the group. | 
| 4/22  |  11:59 PM |  Turn in project proposal on Github |  | 
| 4/29  | 11:59 PM | Finish peer reviewing project proposals  |   |
| 5/3  | 3 PM  | Think about peer projects and how to improve our own project | Discuss how to improve our own project going forward; talk about Project checkpoint and divide up work among group   |
| 5/9  | 3 PM  | Have most of the project checkpoint work finished. | Go over entire project with group and discuss any changes to be made |
| 5/16  | 1:00 PM  | N/A | Discuss what needs to be done to finish the project checkpoint |
| 5/17  | 12:00 PM  |Luke import data into a notebook and finish data cleaning so others can start eda |   |
| 5/19  | 12:00 PM  | Aditi, Simran, Jasmine have EDA finished, Drake start model selection |   |
| 5/20  | 12:00 PM  | Drake have enough model selection implemented to have foothold for final submission |   |
| 5/20  | 1:00 PM  | Update individual sections based on proposal feedback, data cleaning/EDA finsihed, model selection started | Merge branches and clean up checkpoint notebook for submission  |
| 5/25  | 3 PM  | Finalize any additional EDA based on group feedback |   |
| 6/1  | 3 PM  | Continue model comparisons between decision trees and linear regression. Fine tune most optimum parameters |   |
| 6/3  | 3 PM  | Finalize all models | Go through final notebook and clean up. Add explainations and make sure everything is clear and well laid out  |
| 6/8  | 11:59 PM  | Final Project Due | We're done!  |

# Footnotes
<a name="lorenznote"></a>1.[^](#lorenz): Lorenz, T. (9 Dec 2021) Birds Aren’t Real, or Are They? Inside a Gen Z Conspiracy Theory. *The New York Times*. https://www.nytimes.com/2021/12/09/technology/birds-arent-real-gen-z-misinformation.html<br> 
<a name="admonishnote"></a>2.[^](#admonish): Also refs should be important to the background, not some randomly chosen vaguely related stuff. Include a web link if possible in refs as above.<br>
<a name="sotanote"></a>3.[^](#sota): Perhaps the current state of the art solution such as you see on [Papers with code](https://paperswithcode.com/sota). Or maybe not SOTA, but rather a standard textbook/Kaggle solution to this kind of problem


This is a test.